In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode
from fuzzywuzzy import fuzz
import syllable_tokenizer
from abydos.distance import (IterativeSubString, BISIM, DiscountedLevenshtein, Prefix, LCSstr, MLIPNS, Strcmp95,
MRA, Editex, SAPS, FlexMetric, JaroWinkler, HigueraMico, Sift4, Eudex, ALINE, Covington, PhoneticEditDistance)
from abydos.phonetic import PSHPSoundexFirst, Ainsworth

ModuleNotFoundError: No module named 'syllable_tokenizer'

In [2]:
# Interim Data File Locations
interim_data = '../data/interim/'

In [5]:
df = pd.read_csv(interim_data + 'eda_results.csv')
df.shape

(87550, 3)

In [6]:
df.sample(10)

,name_a,name_b,target
27824,Sydnie,Adina,0
61330,Tasya,Mehrnaz,0
79049,Rodri,Sioni,0
79907,Alasdran,Krzysio,0
31589,Zikica,Olcia,0
34263,Frantisek,Raso,0
64940,Oz,Arju,0
54005,Ninsku,Katrijn,0
70185,Johnny,Osku,0
1965,Bobbie,Roberta,1
